In [ ]:
!pip install tensorflow
!pip install matplotlib
!pip install numpy
!pip install scikit-learn



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import DenseNet121, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint

c:\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

In [3]:
# Instantiate DenseNet121 and exclude the top (classification) layer
densenet_base = DenseNet121(input_shape=input_shape, include_top=False, weights='imagenet', input_tensor=input_tensor)

# Freeze DenseNet layers
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features using DenseNet
densenet_features = densenet_base(input_tensor)

# Use some layers of ResNet
resnet_base = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet', input_tensor=input_tensor)

# Freeze ResNet layers
for layer in resnet_base.layers:
    layer.trainable = False

# Extract features using ResNet
resnet_features = resnet_base(input_tensor)

# Concatenate features from both networks
merged_features = concatenate([densenet_features, resnet_features])

# Add classification layers
x = GlobalAveragePooling2D()(merged_features)
x = Dense(256, activation='relu')(x)
output = Dense(6, activation='softmax')(x)

# Create the hybrid model
hybrid_model = Model(inputs=input_tensor, outputs=output)

In [6]:
# Compile the model
hybrid_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have train_data and validation_data directories with subdirectories for each class
train_data_dir = 'D:/Dataset1/train'
validation_data_dir = 'D:/Dataset1/val'
batch_size = 32

# Use data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Use rescaling for validation set
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle for confusion matrix and predictions
)

# Train the model
num_epochs = 10
history = hybrid_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Found 8640 images belonging to 6 classes.
Found 721 images belonging to 6 classes.
Epoch 1/10
 45/270 [====>.........................] - ETA: 20:26 - loss: 0.2949 - accuracy: 0.8847

KeyboardInterrupt: 

In [6]:
# Mapping indices to disease names
class_names = {
    0: 'Aphids',
    1: 'Army_Worm',
    2: 'Bacterial_Blight',
    3: 'Healthy',
    4: 'Powdery_Mildew',
    5: 'Target_Spot'
}

# Plot training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model on the validation set
validation_loss, validation_accuracy = hybrid_model.evaluate(validation_generator)
print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')

# Make predictions on the validation set
y_true = validation_generator.classes
y_pred_prob = hybrid_model.predict(validation_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# Display classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names.values()))

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot confusion matrix inside the graph
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Normalized Confusion Matrix')
plt.colorbar()
plt.xticks(np.arange(6), labels=class_names.values())
plt.yticks(np.arange(6), labels=class_names.values())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Display values inside the matrix
for i in range(len(class_names)):
    for j in range(len(class_names)):
        plt.text(j, i, f'{cm[i, j]:.2f}', horizontalalignment='center', verticalalignment='center')

plt.show()

# Visualize predictions for 10 random images
num_images_to_visualize = 10
random_indices = np.random.choice(len(validation_generator.filenames), size=num_images_to_visualize, replace=False)

for i in random_indices:
    img_path = validation_data_dir + '/' + validation_generator.filenames[i]
    img = plt.imread(img_path)
    plt.imshow(img)
    true_label = class_names[validation_generator.classes[i]]
    predicted_label = class_names[y_pred[i]]
    plt.title(f'True Label: {true_label}\nPredicted Label: {predicted_label}')
    plt.show()


NameError: name 'history' is not defined

<Figure size 1000x600 with 0 Axes>